In [1]:
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStoriesInstruct")

dataset.save_to_disk("/embedding/v-xingwuchen/ts_data/TinyStories/dataset/TinyStoriesInstruct")

/embedding/v-xingwuchen/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 1223.72it/s]
Generating train split: 21755681 examples [00:34, 632907.12 examples/s]
Generating validation split: 218380 examples [00:00, 694725.73 examples/s]
Saving the dataset (1/1 shards): 100%|██████████| 218380/218380 [00:00<00:00, 598209.92 examples/s]


In [2]:
dataset = load_dataset("roneneldan/TinyStories")

dataset.save_to_disk("/embedding/v-xingwuchen/ts_data/TinyStories/dataset/TinyStories")

Repo card metadata block was not found. Setting CardData to empty.
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 816.49it/s]
Generating train split: 2119719 examples [00:05, 372021.96 examples/s]
Generating validation split: 21990 examples [00:00, 382633.86 examples/s]
Saving the dataset (1/1 shards): 100%|██████████| 21990/21990 [00:00<00:00, 100107.18 examples/s]


In [1]:
from transformers import GPT2Model, GPT2Config,GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/embedding/v-xingwuchen/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_from_disk
dataset = load_from_disk("/embedding/v-xingwuchen/ts_data/TinyStories/dataset/TinyStories")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [4]:
for key in dataset["train"][0]:
    print(f"{key.upper()}: {dataset['train'][0][key]}")

TEXT: One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.


In [5]:
from transformers import GPT2LMHeadModel, GPT2Config

config = GPT2Config(
    vocab_size=50257,  # Vocabulary size of the GPT-2 model
    n_embd=64,  # Hidden size of the transformer embeddings
    n_layer=8,  # Number of transformer layers
    n_head=16,  # Number of attention heads
    n_positions=2048,  # Maximum sequence length
)

model = GPT2LMHeadModel(config)

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 64)
    (wpe): Embedding(2048, 64)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPT2Block(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=64, out_features=50257, bias=False)
)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
device = "cuda"
model.to(device)

optim = torch.optim.Adam(model.parameters(), lr=1e-3)

# %%

train_loader = DataLoader(dataset['train'], batch_size=8, shuffle=True)

# %%

tokenizer.pad_token = tokenizer.eos_token

for epoch in range(10):
    for batch in tqdm(train_loader):
        optim.zero_grad()

        tokenized = tokenizer(batch['text'], padding=True, return_tensors='pt',max_length = 512,truncation = True)['input_ids'].to(device)
        logits = model(tokenized)["logits"]
        # print(logits.size())
        # flatten out seq dim
        loss = F.cross_entropy(logits.view(-1, logits.shape[-1]), tokenized.view(-1))
        # tqdm.write(f"Loss: {loss.item()}")
        loss.backward()
        optim.step()
        del tokenized,logits,loss
        torch.cuda.empty_cache()

  1%|▏         | 3953/264965 [07:19<8:26:25,  8.59it/s] 

In [ ]:
print(
    tokenizer.encode("Mi estas Julien.")
)

[41541, 1556, 292, 5979, 2013, 13]
